<a href="https://colab.research.google.com/github/lucasleao03/GSI073---Topicos-Especiais-de-Inteligencia-Artificial-LLMs-Large-Language-Models-/blob/main/GSI073_aula0_support_vector_machine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GSI073 - Tópicos Especiais de Inteligência Artificial

Neste notebook, um tipo de Support Vector Machine Linear.


## Preparação dos dados

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Preparar o dataset
iris = datasets.load_iris()
X = iris.data; y = iris.target

# Dividir em treino e em teste: 70% treino e 30% teste
X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.3, random_state=42)

X = X[y != 1] ; y = y[y != 1] # versicolor
y = torch.tensor(y, dtype=torch.float32)
y[y == 0] = -1  # SVM espera rótulos em {-1, +1}

X = torch.tensor(X, dtype=torch.float32) # Tensor é um tipo especial que suporta muitas dimensões

X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

A nossa Support Vector Machine é basicamente um hiperplano definido por *w* e *b* que melhor separa as classes.

In [ ]:
# Definir parâmetros treináveis da Support Vector Machine: w e b
n_features = X.shape[1]
w = torch.randn(n_features, 1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# === Hiperparâmetros ===
learning_rate = 0.01
epochs = 1000
optimizer = optim.Adam([w, b], lr=learning_rate)

## Execução do treinamento

In [ ]:
for epoch in range(epochs):
    optimizer.zero_grad()

    y_pred = X_train @ w + b  # Modelo SVM (um hiperplano que depende de w e b)

    # Hinge loss: max(0, 1 - y_i * (w^T x_i + b))
    perda_de_classificacao = torch.clamp(1 - y_train.view(-1, 1) * y_pred, min=0).mean()

    # Termo de regularização
    perda_de_distancia_entre_classes = 0.5 * torch.sum(w ** 2) # 2/||w|| é a distância que queremos que seja a maior possível

    # Função objetivo tradicional: minimizar reg + C * hinge
    loss = perda_de_distancia_entre_classes + perda_de_classificacao

    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss={loss.item():.4f}")

In [ ]:
with torch.no_grad():
    y_scores = X_test @ w + b
    y_pred = torch.sign(y_scores)

# Converter para numpy
y_true = y_test.numpy()
y_pred = y_pred.numpy()

# Converter {-1, +1} → {0, 1}
y_true = (y_true > 0).astype(int)
y_pred = (y_pred > 0).astype(int)

# Calcular métricas
acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred)
rec = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
cm = confusion_matrix(y_true, y_pred)

# Exibir resultados formatados
print("\n📊 RESULTADOS DO MODELO:")
print("────────────────────────────")
print(f"✅ Acurácia : {acc*100:.2f}%")
print(f"🎯 Precisão : {prec*100:.2f}%")
print(f"🔍 Recall   : {rec*100:.2f}%")
print(f"⚖️  F1-Score : {f1*100:.2f}%")
print("────────────────────────────")
print("🧩 Matriz de confusão:")
print(cm)
print("────────────────────────────")
print("Legenda: [[TN, FP], [FN, TP]]")
